In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/home/gregory_eritsyan/VESUVIUS_Challenge/jupyter notebooks
Current path:/home/gregory_eritsyan/VESUVIUS_Challenge


In [2]:
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Scrolls_Dataset_Module import Scrolls_Dataset
import matplotlib.patches as patches
from lit_models.UNET_monai_lit import UNET_lit

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


2023-04-15 23:46:08,835 - Created a temporary directory at /tmp/tmp8ut19664
2023-04-15 23:46:08,836 - Writing /tmp/tmp8ut19664/_remote_module_non_scriptable.py


In [3]:
# Initiating Dataset with parameters

# buffer =   -- x,y patchsize for training
# z_start =  --  Offset of slices in the z direction
# z_dim =    -- Number of slices in the z direction. Max value is (64 - z_start)
# validation_rect =  -- rectangle removed for validation set
# shared_height = -- Height to resize all scrolls
# scroll_fragments = -- scrolls to be used 

dataset = Scrolls_Dataset(
                monai= True,
    
                buffer = 64,
                 z_start = 20,
                 z_dim = 30,
                 validation_rect = (1100, 3500, 700, 950),
                shared_height = 8000,
                 downsampling =None,
                 scroll_fragments = [1],#[1,2,3],
                 stage = 'train',
                 shuffle=True,
                 batch_size=128,
                 num_workers =8 ,
                 on_gpu= False,
                          
                         )


In [4]:
lit_model = UNET_lit(z_dim = dataset.z_dim)

wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [5]:
trainer = pl.Trainer(
        accelerator='gpu',
        benchmark=True,
        max_epochs=200,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/'),
        log_every_n_steps=1,
        
        overfit_batches=0,
        precision=16,
     accumulate_grad_batches=4,
        #strategy="ddp" if devices > 1 else None,
    )

trainer.fit(lit_model, datamodule=dataset)

2023-04-15 23:46:15,499 - Using 16bit native Automatic Mixed Precision (AMP)
2023-04-15 23:46:15,548 - GPU available: True (cuda), used: True
2023-04-15 23:46:15,598 - TPU available: False, using: 0 TPU cores
2023-04-15 23:46:15,598 - IPU available: False, using: 0 IPUs
2023-04-15 23:46:15,599 - HPU available: False, using: 0 HPUs


30it [00:16,  1.79it/s]


2023-04-15 23:46:36,939 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2023-04-15 23:46:36,944 - 
  | Name    | Type       | Params
---------------------------------------
0 | metrics | ModuleDict | 0     
1 | model   | UNet       | 1.6 M 
2 | loss    | MaskedLoss | 0     
---------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
3.266     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/gregory_eritsyan/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [6]:
# save model
torch.save(model.state_dict(), 'kaggle/Resnet34_6trans_160height_1.pth')

NameError: name 'model' is not defined